In [ ]:
# CSC Anode thresholds for pp (loose, nominal, tight)
# ME1/1
#140, 140, 140,

# ME1/2
#140, 140, 140,

# ME1/3
#14, 14, 18,

# ME2/1
#28, 56, 58,

# ME2/2
#28, 28, 32,

# ME3/1
#26, 55, 57,

# ME3/2
#26, 26, 34,

# ME4/1
#31, 62, 64,

# ME4/2
#13, 27, 31

In [2]:
import awkward as ak
from coffea import processor
from coffea.nanoevents.methods import candidate
import hist
import pandas as pd
import numpy as np
import pathlib
import shutil 
import os
import matplotlib.pyplot as plt
import mplhep as hep
import warnings
warnings.filterwarnings('ignore')

In [13]:
class MyProcessor(processor.ProcessorABC):
    def __init__(self):
        pass

    def process(self, events):
        dataset = events.metadata['dataset']
        info = ak.zip(
            {
                "runNum": events.runNum,
                "lumiNum": events.lumiNum,
                "evtNum": events.eventNum,
                "nevt": events.eventCounter,
                "ishlt":events.isHLT_PPZeroBias,
            }
        )
        
        ca4 = ak.zip(
            {
                'ncsc':events.nca4CSCcluster,
                'eta':events.ca4CSCclusterEta,
                'phi':events.ca4CSCclusterPhi,
                'x':events.ca4CSCclusterX,        
                'y':events.ca4CSCclusterY,
                'z':events.ca4CSCclusterZ,                
                'size':events.ca4CSCclusterSize, # need for HLT numerator
                'time':events.ca4CSCclusterTime,
                'timeSpread':events.ca4CSCclusterTimeSpread,            
                "nME11_12": events.ca4CSCclusterME11_12,
                'time':events.ca4CSCclusterTime,            
                "nStation10": events.ca4CSCclusterNstation10,
                "avgStation10": events.ca4CSCclusterAvgStation10,            
            }
        )
        
        elctHMT = ak.zip(
            {
                'bits': events.elctHMT_bits,
                'WireNHits': events.elctHMT_WireNHits,
                'sr': events.elctHMT_sr,
            }
        )
        
        run_cut = (info.runNum == 359694)
        # run_cut = (info.runNum == 359998) # for a single file testing
        # elctHMT = elctHMT[run_cut]
        
        hltflag = info.ishlt
        presel = run_cut & hltflag

        
        elctHMT = elctHMT[presel]
        nevents = ak.num(elctHMT, axis=0)
        arr_nevents = np.zeros_like(np.arange(nevents))
        
        ## Threshold array
        #arr = np.concatenate((np.arange(10, 25), np.arange(25, 76, 5)))
        arr = np.array([140, 140, 14, 56, 28, 55, 26, 62, 27])
        names = ['ME11', 'ME12', 'ME13', 'ME21', 'ME22', 'ME31', 'ME32', 'ME41', 'ME42']

        sta_axis = hist.axis.StrCategory(names, growth=True)
        evt_axis = hist.axis.Regular(2, 0, 2, name="flags")
        thr_axis = hist.axis.IntCategory(arr)

        h_nevt = hist.Hist(sta_axis, thr_axis, evt_axis, label="nevt")
        
        ME11_sr = (elctHMT.sr==8) | (elctHMT.sr==9)
        ME12_sr = (elctHMT.sr==7) | (elctHMT.sr==10)
        ME13_sr = (elctHMT.sr==6) | (elctHMT.sr==11)

        ME21_sr = (elctHMT.sr==5) | (elctHMT.sr==12)
        ME22_sr = (elctHMT.sr==4) | (elctHMT.sr==13)

        ME31_sr = (elctHMT.sr==3) | (elctHMT.sr==14)
        ME32_sr = (elctHMT.sr==2) | (elctHMT.sr==15)

        ME41_sr = (elctHMT.sr==1) | (elctHMT.sr==16)
        ME42_sr = (elctHMT.sr==0) | (elctHMT.sr==17)
        
        for i, region in enumerate([ME11_sr,ME12_sr,ME13_sr,ME21_sr,ME22_sr,ME31_sr,ME32_sr,ME41_sr,ME42_sr]):
            sel = ak.any(elctHMT.bits >= 2, axis=1) & ak.any(elctHMT.WireNHits > arr[i], axis=1)
            tmp = region & sel
            l1fired = ak.count_nonzero(tmp)
            nl1fired = np.ones_like(np.arange(l1fired))
            
            h_nevt.fill(names[i], arr[i], arr_nevents)
            h_nevt.fill(names[i], arr[i], nl1fired)
            #print(l1fired)
                
#         # remove empty sub-array 
#         for thres in arr:
#             csc_threshold = ak.any(elctHMT.WireNHits > thres, axis=1)
#             tmp = elctHMT[(ak.num(elctHMT, axis=1) > 0) & (ak.any(elctHMT.bits >= 2, axis=1)) & (csc_threshold)]

#             ME11_sr = (tmp.sr==8) | (tmp.sr==9)
#             ME12_sr = (tmp.sr==7) | (tmp.sr==10)
#             ME13_sr = (tmp.sr==6) | (tmp.sr==11)

#             ME21_sr = (tmp.sr==5) | (tmp.sr==12)
#             ME22_sr = (tmp.sr==4) | (tmp.sr==13)

#             ME31_sr = (tmp.sr==3) | (tmp.sr==14)
#             ME32_sr = (tmp.sr==2) | (tmp.sr==15)

#             ME41_sr = (tmp.sr==1) | (tmp.sr==16)
#             ME42_sr = (tmp.sr==0) | (tmp.sr==17)
        
#             for i, region in enumerate([ME11_sr,ME12_sr,ME13_sr,ME21_sr,ME22_sr,ME31_sr,ME32_sr,ME41_sr,ME42_sr]):
#                 l1fired = ak.count_nonzero(ak.any(region, axis=1))
#                 nl1fired = np.ones_like(np.arange(l1fired))
                
#                 #print(thres, nevents, l1fired)
                
#                 h_nevt.fill(names[i], thres, arr_nevents)
#                 h_nevt.fill(names[i], thres, nl1fired)
               
        return {
            'h_nevt': h_nevt,
        }

    def postprocess(self, accumulator):
        return accumulator

In [14]:
import time
from coffea.nanoevents import BaseSchema

tstart = time.time()

futures_run = processor.Runner(
    executor = processor.FuturesExecutor(compression=None, workers=12),
    schema=BaseSchema,
    maxchunks=50000,
)

fileset = {
    #'run2022c': open("ppZeroBias_Run2022C.txt").read().split("\n"),
    'run2022e': open("ppZeroBias_Run2022E_hltinclude.txt").read().split("\n")
}

# fileset = {
#     'test': [
#         './plots_487.root',
#     ],
# }

output = futures_run(
    fileset,
    treename="simpleCSCshowerFilter/hmt",
    processor_instance=MyProcessor(),
)

elapsed = time.time() - tstart

print(output)

Output()

Output()

{'h_nevt': Hist(
  StrCategory(['ME11', 'ME12', 'ME13', 'ME21', 'ME22', 'ME31', 'ME32', 'ME41', 'ME42'], growth=True, label='Axis 0'),
  IntCategory([140, 140, 14, 56, 28, 55, 26, 62, 27], label='Axis 1'),
  Regular(2, 0, 2, name='flags'),
  storage=Double()) # Sum: 4448874.0}


In [15]:
import pickle
filename = 'rate_histogram_pp_thresholds_v4_hltpath.pickle'
# filename = 'test.pickle'
outfile = open(filename, 'wb')
pickle.dump(output, outfile)
outfile.close()

In [16]:
output['h_nevt']['ME11', 0, :].values()[0]

494306.0

In [17]:
output['h_nevt']['ME11', 0, :].values()[1]

0.0

In [18]:
output['h_nevt']['ME12', 1, :].values()[1]

0.0

In [19]:
output['h_nevt']['ME13', 2, :].values()[1]

34.0

In [20]:
output['h_nevt']['ME21', 3, :].values()[1]

0.0

In [21]:
output['h_nevt']['ME22', 4, :].values()[1]

33.0

In [22]:
output['h_nevt']['ME31', 5, :].values()[1]

4.0

In [23]:
output['h_nevt']['ME32', 6, :].values()[1]

23.0

In [24]:
output['h_nevt']['ME41', 7, :].values()[1]

1.0

In [25]:
output['h_nevt']['ME42', 8, :].values()[1]

25.0